In [1]:
from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter, CharSwapAugmenter, BackTranslationAugmenter
import warnings
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from typing import List
warnings.filterwarnings("ignore")

In [2]:
prompt = "I am an example, we will explore textattack's augmentation capabilities with me."

In [3]:
wn_aug = WordNetAugmenter()
emb_aug = EmbeddingAugmenter()
bt_aug = BackTranslationAugmenter()
easy_aug = EasyDataAugmenter()
char_aug = CharSwapAugmenter()

[nltk_data] Downloading package omw-1.4 to /Users/opop/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/opop/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
wn_augmented_text = wn_aug.augment(prompt)
emb_augmented_text = emb_aug.augment(prompt)
easy_augmented_text = easy_aug.augment(prompt)
char_augmented_text = char_aug.augment(prompt)
bt_augmented_text = bt_aug.augment(prompt)

print(f"Original: {prompt}")
print(f"WordNetAugmenter: {wn_augmented_text[0]}")
print(f"EmbeddingAugmenter: {emb_augmented_text[0]}")
print(f"EasyDataAugmenter: {easy_augmented_text[0]}")
print(f"CharSwapAugmenter: {char_augmented_text[0]}")
print(f"BackTranslationAugmenter: {bt_augmented_text[0]}")

Original: I am an example, we will explore textattack's augmentation capabilities with me.
WordNetAugmenter: iodine am an example, we will explore textattack's augmentation capabilities with me.
EmbeddingAugmenter: I am an example, we will explore textattack's widen capabilities with me.
EasyDataAugmenter: I am an example, we will explore augmentation capabilities with me.
CharSwapAugmenter: I am an example, we will explore textattack's augmentation oapabilities with me.
BackTranslationAugmenter: I'm a copy, exploring the ability to increase testosterone with me.


Le problème du package `textattack` c'est qu'il n'y a pas autant d'options par rapport à `nlpaug`, de plus sur certaines méthodes on ne nous laisse pas beaucoup de choix sur les modèles

`nlpaug` offre de nombreuses méthode pour augmenter le texte, incluant celles vu ci-dessus. Ici, nous allons nous intéresser à l'augmentation via des modèles d'embeddings.

In [15]:
from nlpaug.augmenter.word import WordEmbsAug, ContextualWordEmbsAug